# ECS659U/P/7026P Coursework – 
# The problem
CIFAR-10 Classification
    - classify every image in terms of 1 out of 10 classes

Task: 
Build a model on the training set and evaluate it on the test set.

Implement a specific model:
The model:
    An architecture to process images based on CNN
    Backbone (B1,...,Bn) and classifier

Backbone: 
    Consist of N blocks
    
    Block: minimum implementation
        1 Linear/ MLP layer predicting a vector with K elements from input tensor
        K Conv layers which are combined using a to produce a single output O

Classifier:
    Takes input from last block
    computes mean feature 
    passes f to classifier 

    softmax regression classifier or MLP


Taks: 
Read dataset and create dataloaders
create model
create loss and optimiser 
write training script to train the model:
    include  in report:
        curves for evolution of loss
        curves for evolution of training and testing accuracy
        training details including hyper parameters

model accuaracy: 
95> = 20%
95< > 85 = 15%
85% < >80% = 10
80% < > 70% = 5
<70% 0 



In [1]:
#import required libraries 
import torchvision
import torch 
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import math
import os
from IPython import display#
import seaborn as sns 
import matplotlib.pyplot as plt
sns.set()

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Read dataset and create data loader (5%)

In [2]:
#Loading the dataset
## Define the CIFAR 10 Classes
LABELS = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Transaform the dataset by changing the values to Tensors and Normalising the Tensor Values
transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32),
    torchvision.transforms.ColorJitter(),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Batch Sizes: The amount of images used in each epoch
BATCH_SIZE = 256

# Download the CIFAR Dataset 
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create a datset loader 
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
X, y = next(iter(train_loader)) 
print(X.size())
#Check the batch size: returns ([batch_size,3 channels, 32 pixel, 32 pixel])

torch.Size([256, 3, 32, 32])


# The Model (40%)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a single block of the backbone that consists of a convolutional layer, batch normalization,
# ReLU activation, and average pooling.
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ConvBlock, self).__init__()
        # Convolutional layer
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        # Batch normalization
        self.bn = nn.BatchNorm2d(out_channels)
        # ReLU activation
        self.activation = nn.ReLU(inplace=True)
        # Average pooling
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.activation(out)
        out = self.pool(out)
        return out

# Define the backbone, which consists of N blocks. The first block takes an input tensor with input_channels channels
# and outputs a tensor with output_channels channels. The subsequent blocks take an input tensor with output_channels channels
# and output a tensor with output_channels channels.
class Backbone(nn.Module):
    def __init__(self, num_blocks, input_channels, output_channels):
        super(Backbone, self).__init__()
        # First block
        self.block1 = ConvBlock(input_channels, output_channels)
        # Subsequent blocks
        self.blocks = nn.ModuleList(
            [ConvBlock(output_channels, output_channels) for _ in range(num_blocks - 1)]
        )
        
    def forward(self, x):
        out = self.block1(x)
        for block in self.blocks:
            out = block(out)
        return out
    
# Define the classifier, which takes the output of the backbone as input. It applies average pooling to reduce
# the spatial dimensions to 1x1, and then passes the resulting tensor through a fully connected layer.
# If hidden_dim is not None, it adds an additional hidden layer before the output layer.
class Classifier(nn.Module):
    def __init__(self, input_channels, num_classes, hidden_dim=None):
        super(Classifier, self).__init__()
        # Adaptive average pooling
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        if hidden_dim is None:
            # Output layer without hidden layer
            self.fc = nn.Linear(input_channels, num_classes)
        else:
            # Output layer with hidden layer
            self.fc = nn.Sequential(
                        nn.Linear(input_channels, hidden_dim),
                        nn.ReLU(inplace=True),
                        nn.Linear(hidden_dim, num_classes)
                    )
        
    def forward(self, x):
        out = self.pool(x)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

# Define the overall CNN model that combines the backbone and classifier. It takes num_blocks, input_channels,
# output_channels, num_classes, and hidden_dim as input arguments.
class CNN(nn.Module):
    def __init__(self, num_blocks, input_channels, output_channels, num_classes, hidden_dim=None):
        super(CNN, self).__init__()
        # Backbone
        self.backbone = Backbone(num_blocks, input_channels, output_channels)
        # Classifier
        self.classifier = Classifier(output_channels, num_classes, hidden_dim)
        
    def forward(self, x):
        out = self.backbone(x)
        out = self.classifier(out)
        return out


if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CNN(num_blocks=5, input_channels=3, output_channels=64, num_classes=10, hidden_dim=512).to(device)
    print(model)


CNN(
  (backbone): Backbone(
    (block1): ConvBlock(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU(inplace=True)
      (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    )
    (blocks): ModuleList(
      (0-3): 4 x ConvBlock(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU(inplace=True)
        (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
      )
    )
  )
  (classifier): Classifier(
    (pool): AdaptiveAvgPool2d(output_size=(1, 1))
    (fc): Sequential(
      (0): Linear(in_features=64, out_features=512, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=512, out_features=10, bias=True)
    )
  )
)


# create loss and optimiser 5%

In [5]:
import torch.optim as optim

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Define the number of epochs to train for
num_epochs = 30

# Move the model and loss function to the GPU if available
if torch.cuda.is_available():
    model = model.cuda()
    loss_function = loss_function.cuda()

# Train the model
for epoch in range(num_epochs):
    # Set the model to train mode
    model.train()
    
    # Initialize the running loss and accuracy
    running_loss = 0.0
    running_corrects = 0
    
    # Iterate over the training data
    for data, target in train_loader:
        # Move the inputs and labels to the GPU if available
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        loss = loss_function(output, target)
        
        # Backward pass and optimizer step
        loss.backward()
        optimizer.step()
        
        # Update the running loss and accuracy
        _, predictions = torch.max(output, 1)
        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(predictions == target.data)
    
    # Calculate the epoch loss and accuracy
    train_loss = running_loss / len(train_data)
    train_acc = running_corrects.double() / len(train_data)
    
    # Print the epoch loss and accuracy
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}'.format(epoch+1, num_epochs, train_loss, train_acc))
    
    # Set the model to evaluation mode
    model.eval()
    
    # Initialize the running loss and accuracy
    running_loss = 0.0
    running_corrects = 0
    
    # Iterate over the test data
    for data, target in test_loader:
        # Move the inputs and labels to the GPU if available
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        # Forward pass
        output = model(data)
        loss = loss_function(output, target)
        
        # Update the running loss and accuracy
        _, predictions = torch.max(output, 1)
        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(predictions == target.data)
    
    # Calculate the epoch loss and accuracy
    test_loss = running_loss / len(test_data)
    test_acc = running_corrects.double() / len(test_data)
    
    # Print the epoch loss and accuracy
    print('Epoch [{}/{}], Test Loss: {:.4f}, Test Acc: {:.4f}'.format(epoch+1, num_epochs, test_loss, test_acc))


Epoch [1/30], Train Loss: 1.3533, Train Acc: 0.5024
Epoch [1/30], Test Loss: 1.2527, Test Acc: 0.5717
Epoch [2/30], Train Loss: 0.9275, Train Acc: 0.6680
Epoch [2/30], Test Loss: 1.1055, Test Acc: 0.6075
Epoch [3/30], Train Loss: 0.7807, Train Acc: 0.7244
Epoch [3/30], Test Loss: 0.9413, Test Acc: 0.6766


In [ ]:
import matplotlib.pyplot as plt

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the number of epochs to train for
num_epochs = 30

# Move the model and loss function to the GPU if available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

# Initialize the lists to store the training and test losses and accuracies
train_losses = []
test_losses = []
train_accs = []
test_accs = []

# Train the model
for epoch in range(num_epochs):
    # Set the model to train mode
    model.train()
    
    # Initialize the running loss and accuracy
    train_running_loss = 0.0
    train_running_corrects = 0
    
    # Iterate over the training data
    for train_inputs, train_labels in train_loader:
        # Move the inputs and labels to the GPU if available
        if torch.cuda.is_available():
            train_inputs = train_inputs.cuda()
            train_labels = train_labels.cuda()
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        train_outputs = model(train_inputs)
        train_loss = criterion(train_outputs, train_labels)
        
        # Backward pass and optimizer step
        train_loss.backward()
        optimizer.step()
        
        # Update the running loss and accuracy
        _, train_preds = torch.max(train_outputs, 1)
        train_running_loss += train_loss.item() * train_inputs.size(0)
        train_running_corrects += torch.sum(train_preds == train_labels.data)
    
    # Calculate the epoch loss and accuracy
    train_loss_epoch = train_running_loss / len(train_data)
    train_acc_epoch = train_running_corrects.double() / len(train_data)
    
    # Append the training loss and accuracy to the lists
    train_losses.append(train_loss_epoch)
    train_accs.append(train_acc_epoch)
    
    # Print the epoch loss and accuracy
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}'.format(epoch+1, num_epochs, train_loss_epoch, train_acc_epoch))
    
    # Set the model to evaluation mode
    model.eval()
    
    # Initialize the running loss and accuracy
    test_running_loss = 0.0
    test_running_corrects = 0
    
    # Iterate over the test data
    for test_inputs, test_labels in test_loader:
        # Move the inputs and labels to the GPU if available
        if torch.cuda.is_available():
            test_inputs = test_inputs.cuda()
            test_labels = test_labels.cuda()
        
        # Forward pass
        test_outputs = model(test_inputs)
        test_loss = criterion(test_outputs, test_labels)
        
        # Update the running loss and accuracy
        _, test_preds = torch.max(test_outputs, 1)
        test_running_loss += test_loss.item() * test_inputs.size(0)
        test_running_corrects += torch.sum(test_preds == test_labels.data)
    
    # Calculate the epoch loss and accuracy
    test_loss_epoch = test_running_loss / len(test_data)
    test_acc_epoch = test_running_corrects.double() / len(test_data)
    
    # Append the test loss and accuracy to the lists
    test_losses.append(test_loss_epoch)
    test_accs.append(test_acc_epoch)
    
    # Print the epoch loss and accuracy
    print('Epoch [{}/{}], Test Loss: {:.4f}, Test Acc: {:.4f}'.format(epoch+1, num_epochs, test_loss_epoch, test_acc_epoch))


In [ ]:
plt.plot(range(num_epochs), train_losses, label='Train Loss')
plt.plot(range(num_epochs), test_losses, label='Test Loss')
plt.title('Loss Evolution')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
print(train_acc.shape)

In [ ]:
import matplotlib.pyplot as plt


# Plot the training and test losses against the number of epochs
plt.plot(range(num_epochs), train_losses, label='Train Loss')
plt.plot(range(num_epochs), test_losses, label='Test Loss')

# Add labels and legend
plt.title('Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot the training and testing accuracy curves
plt.plot(train_accs, label='Training Accuracy')
plt.plot(test_accs, label='Testing Accuracy')
plt.title('Accuaracy evolution')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Calculate the final test accuracy
model.eval()  # Set the model to evaluation mode
running_corrects = 0  # Initialize the number of correct predictions to 0

# Iterate over the test data
for data, target in test_loader:
    if torch.cuda.is_available():
        data = data.cuda()  # Move the inputs to the GPU if available
        target = target.cuda()  # Move the labels to the GPU if available
    output = model(data)  # Forward pass
    _, predictions = torch.max(output, 1)  # Get the predicted classes
    running_corrects += torch.sum(predictions == target.data)  # Count the number of correct predictions
    
final_test_acc = running_corrects.double() / len(test_data)  # Calculate the final test accuracy

# Print the final test accuracy
print('Final Test Accuracy: {:.4f}'.format(final_test_acc))
